In [ ]:
### Import Libraries.

import scanpy as sc
import anndata as ad
import palantir
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
from numba.core.errors import NumbaDeprecationWarning

warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)
warnings.filterwarnings("ignore", module="scanpy", message="No data for colormapping")

In [ ]:
### Load Data.

os.chdir("/folder/")
adata = ad.read_h5ad('adata.h5ad')

In [ ]:
### Diffusion Map and MAGIC Imputation.

dm_res = palantir.utils.run_diffusion_maps(adata, n_components = 20)
ms_data = palantir.utils.determine_multiscale_space(adata)
imputed_X = palantir.utils.run_magic_imputation(adata, dm_res)

In [ ]:
### Plots of Imputed Expression Across Embeddings.

sc.settings.set_figure_params(dpi = 800, figsize = (4, 4))
sc.pl.embedding(adata, basis = "X_umap", color = "Cluster_Column", frameon = False)
sc.pl.embedding(adata, basis = "X_pca", color = "Cluster_Column", frameon = False)
sc.pl.embedding(adata, basis = "X_met_embedding", color = "Cluster_Column", frameon = False)

In [ ]:
### Plots of Selected Gene Sets.

genesets = [
    ["Feature_1", "Feature_2", "Feature_3", "Feature_4"],
    ["Feature_5", "Feature_6", "Feature_7", "Feature_8"]
]
for i, g in enumerate(genesets, 1):
    sc.pl.embedding(
        adata, basis = "X_pca", color = g, frameon = False, ### X_umap, X_met_embedding
        save = f"_magic_plot_{i}.png"
    )

In [ ]:
### Run Palantir Pseudotime and Entropy Inference.

terminal_states = pd.Series(
    ["Cluster_1", "Cluster_2"],
    index = [
        "End_Cell_Cluster_1",
        "End_Cell_Cluster_2"
    ]
)

pr_res = palantir.core.run_palantir(
    adata,
    early_cell = "Start_Cell",
    terminal_states = terminal_states,
    knn = 30
)

In [ ]:
### Summarize Pseudotime and Entropy per Cluster.

mean_entropy = adata.obs.groupby("Cluster_Column")["palantir_entropy"].mean().reset_index()
mean_pseudotime = adata.obs.groupby("Cluster_Column")["palantir_pseudotime"].mean().reset_index()

In [ ]:
### Boxplots of Entropy and Pseudotime.

plt.figure(figsize = (10, 5))
sns.boxplot(x = "Cluster_Column", y = "palantir_entropy", data = adata.obs)
plt.xticks(rotation = 45)
plt.title("Entropy across Cluster_Column")
plt.show()

plt.figure(figsize = (10, 5))
sns.boxplot(x = "Cluster_Column", y = "palantir_pseudotime", data = adata.obs)
plt.xticks(rotation = 45)
plt.title("Pseudotime across Cluster_Column")
plt.show()

In [ ]:
### Plot Trajectories and Gene Trends.

palantir.plot.plot_palantir_results(adata, s = 4, embedding_basis = "X_pca") ### X_umap, X_met_embedding
palantir.plot.plot_trajectory(adata, "Cluster_1", cell_color = "palantir_pseudotime")
genes = ["Feature_1", "Feature_2", "Feature_3", "Feature_4"]
palantir.plot.plot_gene_trends(adata, genes)
palantir.plot.plot_gene_trend_heatmaps(adata, genes)